Explanation: 
This code firstly calculates the thermophysical properties of the relevant vegetables, before calculating the cooling time when 20kg of each vegetable is put into the cold storage at 07:00 and 08:00. 

In [1]:
import CoolProp.CoolProp as CP
import math
import numpy as np
from scipy.special import j0, j1

# %%
vegetables = [
    {
        "name": "tomatoes",
        "Protein": 0.0085,
        "Fat": 0.0033,
        "Carbs": 0.0464,
        "Fiber": 0.011,
        "Ash": 0.0042,
        "Water": 0.9376,
        "T_wanted": 273 + 13,
        "d": 0.056,
        
    },
    {
        "name": "bell peppers",
        "Protein": 0.01,
        "Fat": 0.003,
        "Carbs": 0.06,
        "Fiber": 0.021,
        "Ash": 0.0087,
        "Water": 0.92,
        "T_wanted": 273 + 10,
        "d": 2*0.01, #Times 2 as the half thickness does not apply here
        
    },
    { 
       "name": "hot peppers",
        "Protein" : 0.0187,
        "Fat" : 0.0044,
        "Carbs" : 0.0881,
        "Fiber" : 0.015,
        "Ash" : 0.0087,
        "Water" : 0.88,
        "T_wanted" : 273 +  10,
        "d" : 0.0075*2, #Times 2 as the half thickness does not apply here
        
        

    },
    {
       "name": "lettuce",
        "Protein" : 0.01,
        "Fat" : 0.0019,
        "Carbs" : 0.0209,
        "Fiber" : 0.014,
        "Ash" : 0.0048,
        "Water" : 0.9589,
        "T_wanted" : 273 +  1,
        "d" : 0.17, 
         
    },
    {
        "name": "cabbage",
        "Protein" : 0.0144,
        "Fat" : 0.0027,
        "Carbs" : 0.0543,
        "Fiber" : 0.023,
        "Ash" : 0.00071,
        "Water" : 0.9215,
        "T_wanted" : 273 +  1,
        "d" : 0.17, 
         
    },
    {
        "name": "cucumber",
        "Protein" : 0.0069,
        "Fat" : 0.0013,
        "Carbs" : 0.0276,
        "Fiber" : 0.008,
        "Ash" : 0.0041,
        "Water" : 0.9601,
        "T_wanted" : 273 +  10,
        "d" : 0.04, 
        
    }

]

# Time with respective temperatures, change for different temperatures and the cooling time changes. 
time = {
    "07": 273 + 28.3,
    "08": 273 + 29.7,
}

# Calculating Theta for each vegetable and time
def cooling(veg, T):
    L = veg["d"] / 2; #carachteristic length 
    T_mean = (T - veg["T_wanted"]) / 2;
    
    # Thermal conductivities
    k = [0]*7;
    k[1] = veg["Protein"] * (0.17881 + (1.1958 * T_mean * 10 ** -3) - (2.7178 * 10 ** -6 * T_mean ** 2));
    k[2] = veg["Fat"] * (0.18071 - (2.7604 * T_mean * 10 ** -4) - (1.7749 * 10 ** -7 * T_mean ** 2));
    k[3] = (veg["Carbs"] - veg["Fiber"]) * (0.20141 + 1.3874 * 10 ** -3 * T_mean - 4.3312 * 10 ** -6 * T_mean ** 2);
    k[4] = veg["Fiber"] * (0.18331 + 1.2497 * 10 ** -3 * T_mean - 3.1683 * 10 ** -6 * T_mean ** 2);
    k[5] = veg["Ash"] * (0.329620 + 1.4011 * 10 ** -3 * T_mean - 2.9069 * 10 ** -6 * T_mean ** 2);
    k[6] = veg["Water"] * (0.57109 + 1.7625 * 10 ** -3 * T_mean - 6.7036 * 10 ** -6 * T_mean ** 2);
    k_total = sum(k[1:7]);

    # Specific heat capacity
    cp = [0] * 7;
    cp[1] = veg["Protein"] * (2.0082 + 1.2089e-3 * T_mean - 1.3129e-6 * T_mean ** 2);
    cp[2] = veg["Fat"] * (1.9842 + 1.4733e-3 * T_mean - 4.8008e-6 * T_mean ** 2);
    cp[3] = (veg["Carbs"] - veg["Fiber"]) * (1.5488 + 1.9625e-3 * T_mean - 5.9399e-6 * T_mean ** 2);
    cp[4] = veg["Fiber"] * (1.8459 + 1.8306e-3 * T_mean - 4.6509e-6 * T_mean ** 2);
    cp[5] = veg["Ash"] * (1.0926 + 1.8896e-3 * T_mean - 3.6817e-6 * T_mean ** 2);
    cp[6] = veg["Water"] * (4.1289 - 9.0864e-5 * T_mean + 5.4731e-6 * T_mean ** 2);
    cp_total = sum(cp[1:7]) * 1000

    # Densities
    rho = [0] * 7;
    rho[1] = veg["Protein"] * (1.3299 * 10**3 - 5.1840 * 10**-1 * T_mean);
    rho[2] = veg["Fat"] * (9.2559 * 10**2 - 4.1757 * 10**-1 * T_mean);
    rho[3] = (veg["Carbs"] - veg["Fiber"]) * (1.5991 * 10**3 - 3.1046 * 10**-1 * T_mean);
    rho[4] = veg["Fiber"] * (1.3115 * 10**3 - 3.6589 * 10**-1 * T_mean);
    rho[5] = veg["Ash"] * (2.4238 * 10**3 - 2.8063 * 10**-1 * T_mean);
    rho[6] = veg["Water"] * (9.9718 * 10**2 + 3.1439 * 10**-3 * T_mean - 3.7574 * 10**-3 * T_mean ** 2);
    rho_total = sum(rho[1:7])

    #print("k:",k_total);
    #print("Cp:",cp_total);
    #print("Rho:",rho_total);

    alpha = k_total / (cp_total * rho_total); #thermal diffusivity
    L = veg["d"] / 2;
    Bi = 25 * L / k_total;  # assuming h_it = 25 W/m^2*K 
    ln_Bi = math.log(Bi);

    if 0.1 < Bi < 100:
        if veg["name"] in ["tomatoes", "cabbage", "lettuce"]: #Sphere
            
            w = 1.573729 + 0.6429061 * ln_Bi + 0.047859 * (ln_Bi ** 2) - 0.03553 * (ln_Bi ** 3) - 0.004907 * (ln_Bi ** 4) + 0.001563 * (ln_Bi ** 5);
            f = L**2 * math.log(10) / (alpha * w**2);
            j_c = (2 * (math.sin(w) - w * math.cos(w))) / (w - math.sin(w) * math.cos(w));
        
        elif veg["name"] in ["bell peppers"]: #Slab
            u = 0.860972+0.312133*ln_Bi+0.007986*ln_Bi**2-0.016192*ln_Bi**3-0.001190*ln_Bi**4+0.000581*ln_Bi**5;
            j_c = 2*math.sin(u)/(u+math.sin(u)*math.cos(u));
            f = L**2*math.log(10)/(u**2*alpha);
        else:
            #Cylinder
            v = 1.257493 + 0.487941 * ln_Bi + 0.025322 * (ln_Bi ** 2) - 0.026568 * (ln_Bi ** 3) - 0.002888 * (ln_Bi ** 4) + 0.001078 * (ln_Bi ** 5);
            f = L**2 * math.log(10) / (alpha * v ** 2);
            j_c = (2 * j1(v)) / ((v * j0(v)) - j1(v) ** 2);
        
        Y = (veg["T_wanted"] - 268) / (T - 268);
        Theta = round((-f / 2.303) * math.log(Y / j_c), 2);
        
        #print(f"Specific heat capacity of {veg['name']} is {cp_total:.2f} J/(kg·K)")
        return Theta, cp_total;  # Time in minutes
        

def calculate_heat_removal():
    heat_removed = 0
    for veg in vegetables:
        for time_key, T in time.items():
            cooling_time, Cp = cooling(veg, T)
            mass = 20 #kg

            delta_T = T - veg["T_wanted"]  # Celcius
            Q = mass * Cp * delta_T/(cooling_time*1000)  #KJ/s

            print(f"Time to cool {veg['name']} at time {time_key} is {cooling_time/60:.2f} minutes");


calculate_heat_removal()

Time to cool tomatoes at time 07 is 29.15 minutes
Time to cool tomatoes at time 08 is 30.45 minutes
Time to cool bell peppers at time 07 is 27.15 minutes
Time to cool bell peppers at time 08 is 28.44 minutes
Time to cool hot peppers at time 07 is 12.33 minutes
Time to cool hot peppers at time 08 is 12.78 minutes
Time to cool lettuce at time 07 is 328.30 minutes
Time to cool lettuce at time 08 is 333.93 minutes
Time to cool cabbage at time 07 is 329.39 minutes
Time to cool cabbage at time 08 is 335.02 minutes
Time to cool cucumber at time 07 is 46.22 minutes
Time to cool cucumber at time 08 is 47.56 minutes
